In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Cargar el dataset (ajusta la ruta a tu archivo)
filename = '/home/rsa/projects/gpd/data/raw/datasets/analyst_picks.csv'
df = pd.read_csv(filename, sep=";")

# Extraer el código de estación (antes del primer "_")
df.insert(0, "Estacion", df["mseed"].str.split("_").str[0])

# Verificar los primeros registros
df.head()

In [ ]:
# Guardar archivo modificado
OUT_CSV = '/home/rsa/projects/gpd/data/raw/datasets/dataset.csv'
df.to_csv(OUT_CSV, index=False, sep=";", encoding="utf-8-sig")
print("\nGuardado en:", OUT_CSV)

In [ ]:
# Listar estaciones unicas
estaciones = df["Estacion"].unique()
print("Estaciones encontradas:")
print(estaciones)
print("\nNumero total de estaciones:", len(estaciones))
# Contar el total de registros por estacion
conteo_estaciones = df["Estacion"].value_counts()
# Mostrar en tabla
print(conteo_estaciones)

# Graficar
plt.figure(figsize=(10,5))
conteo_estaciones.plot(kind="bar")
plt.title("Total de datos por estación")
plt.xlabel("Estación")
plt.ylabel("Número de registros")
plt.xticks(rotation=45)
plt.show()

**Filtrado**


Filtrado de estaciones con fases ponderadas

In [ ]:
# Eliminar filas donde ambas ponderaciones (P y S) son NA
df_filtrado_ponderado = df.dropna(subset=["Pond T-P", "Pond T-S"], how="all")

# Mostrar número de filas resultantes
print("Total de filas resultantes:", len(df_filtrado_ponderado))

# Listar estaciones unicas
estaciones = df_filtrado_ponderado["Estacion"].unique()
print("Estaciones encontradas:")
print(estaciones)
print("\nNumero total de estaciones:", len(estaciones))
# Contar el total de registros por estacion
conteo_estaciones = df_filtrado_ponderado["Estacion"].value_counts()
# Mostrar en tabla
print(conteo_estaciones)

# Graficar
plt.figure(figsize=(10,5))
conteo_estaciones.plot(kind="bar")
plt.title("Total de datos por estación")
plt.xlabel("Estación")
plt.ylabel("Número de registros")
plt.xticks(rotation=45)
plt.show()


Filtrado de estaciones de 64 Hz:

In [ ]:
# Filtrar filas con Muestreo = 64.0
df_filtrado_64 = df_filtrado_ponderado[df_filtrado_ponderado["Muestreo"] == 64.0]
# Mostrar número de filas resultantes
print("Total de filas resultantes:", len(df_filtrado_64))

Filtrado de estaciones que tengan unicamente la ponderacion P

In [ ]:
# Eliminar filas donde Pond T-S sea NA
df_filtrado_p = df_filtrado_ponderado.dropna(subset=["Pond T-S"])
# Mostrar número de filas resultantes
print("Total de filas resultantes:", len(df_filtrado_p))

Filtrado de estaciones que tengan unicamente la ponderacion S

In [ ]:
# Eliminar filas donde Pond T-P sea NA
df_filtrado_s = df_filtrado_ponderado.dropna(subset=["Pond T-P"])
# Mostrar número de filas resultantes
print("Total de filas resultantes:", len(df_filtrado_s))

In [ ]:
# Filtrar filas con Muestreo = 64.0
df_filtrado = df[df["Muestreo"] == 64.0]

# Eliminar filas donde ambas ponderaciones (P y S) son NA
#df_filtrado = df_filtrado.dropna(subset=["Pond T-P", "Pond T-S"], how="all")

# Eliminar filas donde al menos una ponderacion (P o S) sea NA
df_filtrado_ps = df_filtrado.dropna(subset=["Pond T-P", "Pond T-S"], how="any")

# Eliminar filas donde Pond T-P sea NA
df_filtrado_s = df_filtrado.dropna(subset=["Pond T-P"])

# Eliminar filas donde Pond T-P sea NA
df_filtrado_s = df_filtrado.dropna(subset=["Pond T-S"])

# Mostrar número de filas resultantes
print("Total de filas resultantes:", len(df_filtrado))

# Vista previa
df_filtrado.head()

In [ ]:
# Listar estaciones unicas
estaciones = df_filtrado["Estacion"].unique()
print("Estaciones encontradas:")
print(estaciones)

print("\nNumero total de estaciones:", len(estaciones))

In [ ]:
# Contar el total de registros por estacion
conteo_estaciones = df_filtrado["Estacion"].value_counts()

# Mostrar en tabla
print(conteo_estaciones)

# Graficar
plt.figure(figsize=(10,5))
conteo_estaciones.plot(kind="bar")
plt.title("Total de datos por estación (Muestreo=64.0)")
plt.xlabel("Estación")
plt.ylabel("Número de registros")
plt.xticks(rotation=45)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# ========= Parametros =========
CSV_PATH = filename      # ruta a tu CSV
FREQ = "W"                 # granularidad temporal: "D" (dia), "W" (semana), "M" (mes)
SEED = 42                  # semilla para reproducibilidad
ESTACIONES_OBJ = ["LABR", "CUSH", "CHAI", "UVER", "PORT"]
TARGET_POR_ESTACION = 20  # total por estacion
# ==============================

rng = np.random.RandomState(SEED)

# 1) Cargar y preparar
df = pd.read_csv(CSV_PATH, sep=";")
# Extraer codigo de estacion
df.insert(0, "Estacion", df["mseed"].str.split("_").str[0])

# Si ya trabajas con df_filtrado, sustituye df por df_filtrado a partir de aqui:
# df = df_filtrado.copy()

# Asegurar tipos de tiempo
df["T-ini"] = pd.to_datetime(df["T-ini"], errors="coerce", utc=True)
df["T-fin"] = pd.to_datetime(df["T-fin"], errors="coerce", utc=True)

# 2) Filtrado opcional (descomenta si quieres mantener exactamente el mismo filtrado previo)
df = df[df["Muestreo"] == 64.0]
# Eliminar filas donde ambas ponderaciones (P y S) son NA
#df = df.dropna(subset=["Pond T-P", "Pond T-S"], how="all")
# Eliminar filas donde al menos una ponderacion (P o S) sea NA
df_filtrado = df_filtrado.dropna(subset=["Pond T-P", "Pond T-S"], how="any")

# 3) Quedarse con las estaciones objetivo
df = df[df["Estacion"].isin(ESTACIONES_OBJ)].copy()

# 4) Crear bins temporales (fechas similares por semana/mes/dia segun FREQ)
#    Se usa el inicio del periodo como marca del bin
df["time_bin"] = df["T-ini"].dt.to_period(FREQ).dt.start_time

# 5) Identificar bins comunes a todas las estaciones (interseccion de bins)
bins_por_est = {
    est: set(df.loc[df["Estacion"] == est, "time_bin"].dropna().unique())
    for est in ESTACIONES_OBJ
}
bins_comunes = set.intersection(*bins_por_est.values()) if bins_por_est else set()
bins_comunes = sorted(bins_comunes)

if len(bins_comunes) == 0:
    raise ValueError("No hay bins temporales comunes entre todas las estaciones con la frecuencia seleccionada. "
                     "Prueba con otra FREQ (p.ej., 'M') o relaja filtros.")

# 6) Prealocar cuotas por bin (distribucion lo mas uniforme posible)
#    Ej: si hay 40 semanas comunes y se quieren 200 muestras, asigna 5 por semana, con el resto distribuido
def cuotas_por_bin(target, num_bins):
    base = target // num_bins
    resto = target % num_bins
    cuotas = np.full(num_bins, base, dtype=int)
    # distribuir el resto uno por uno desde el inicio
    cuotas[:resto] += 1
    return cuotas

cuotas_base = cuotas_por_bin(TARGET_POR_ESTACION, len(bins_comunes))

# 7) Funcion de muestreo estratificado por bins para una estacion
def muestrear_por_bins(df_est, bins_ord, cuotas, rng):
    tomadas = []
    # intentar cumplir cuotas por cada bin
    for bin_val, cuota in zip(bins_ord, cuotas):
        if cuota <= 0:
            continue
        candidatos = df_est[df_est["time_bin"] == bin_val]
        if len(candidatos) == 0:
            continue
        # si hay menos que la cuota, tomar todos; si hay mas, sample
        if len(candidatos) <= cuota:
            tomadas.append(candidatos)
        else:
            tomadas.append(candidatos.sample(n=cuota, random_state=rng))
    sel = pd.concat(tomadas) if tomadas else df_est.iloc[0:0]
    # si no alcanza el total, completar con otros bins (fuera de bins comunes) o sobrantes
    faltan = TARGET_POR_ESTACION - len(sel)
    if faltan > 0:
        # prioridad: tomar de otros registros de la misma estacion no seleccionados
        resto = df_est.drop(sel.index)
        if len(resto) > 0:
            if len(resto) <= faltan:
                sel = pd.concat([sel, resto])
            else:
                sel = pd.concat([sel, resto.sample(n=faltan, random_state=rng)])
    return sel.head(TARGET_POR_ESTACION)

# 8) Muestrear 200 por estacion
muestras_estaciones = []
resumen = {}
for est in ESTACIONES_OBJ:
    df_est = df[df["Estacion"] == est].copy()
    sel_est = muestrear_por_bins(df_est, bins_comunes, cuotas_base, rng)
    muestras_estaciones.append(sel_est)
    resumen[est] = len(sel_est)

df_sampled = pd.concat(muestras_estaciones).reset_index(drop=True)

# 9) Validaciones
conteo_final = df_sampled["Estacion"].value_counts().reindex(ESTACIONES_OBJ, fill_value=0)
total_final = len(df_sampled)

print("Resumen por estacion (esperado 200 c/u):")
print(conteo_final)
print("\nTotal esperado:", TARGET_POR_ESTACION * len(ESTACIONES_OBJ))
print("Total obtenido:", total_final)

# 10) (Opcional) inspeccion rápida de alineacion temporal
print("\nPrimeras fechas por estacion en la muestra:")
print(df_sampled.groupby("Estacion")["T-ini"].min())
print("\nUltimas fechas por estacion en la muestra:")
print(df_sampled.groupby("Estacion")["T-ini"].max())

# 11) Guardar resultado
OUT_CSV = "/content/drive/MyDrive/Colab Notebooks/TFM/dataset_estratificado_{}_{}.csv".format(FREQ, TARGET_POR_ESTACION * len(ESTACIONES_OBJ))
df_sampled.to_csv(OUT_CSV, index=False)
print("\nGuardado en:", OUT_CSV)


Esta nueva version:

Para PORT y UVER toma todo lo disponible (sin NA y a 64 Hz) y no fuerza cuotas por semana/mes; luego, si no llegan a 200, rellena con muestras de LABR/CUSH/CHAI, priorizando los mismos bins temporales donde aparecieron PORT/UVER (para mantener “fechas similares”).

Para LABR/CUSH/CHAI si hace una seleccion priorizando los bins de las estaciones pequenas; si falta, completa con cualquier bin.

In [ ]:
import pandas as pd
import numpy as np

# ========= Rutas en Colab =========
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
IN_CSV  = '/content/drive/MyDrive/Colab Notebooks/TFM/test.csv'
OUT_CSV = '/content/drive/MyDrive/Colab Notebooks/TFM/dataset_estratificado.csv'
# ==================================

# ========= Parametros =========
FREQ = "W"                 # "D" dia, "W" semana, "M" mes
SEED = 42
TARGET_POR_EST = 20
ESTACIONES_OBJ = ["LABR", "CUSH", "CHAI", "PORT", "UVER"]
SMALL_STATIONS = {"PORT", "UVER"}
# ==============================

rng = np.random.RandomState(SEED)

# 1) Cargar y preparar
df = pd.read_csv(IN_CSV, sep=";")

# Estacion
df.insert(0, "Estacion", df["mseed"].str.split("_").str[0])

# Tiempos
df["T-ini"] = pd.to_datetime(df["T-ini"], errors="coerce", utc=True)
df["T-fin"] = pd.to_datetime(df["T-fin"], errors="coerce", utc=True)

# 2) Filtros estrictos requeridos
df = df[df["Muestreo"] == 64.0].copy()
# Sin NA en ponderaciones: ambas deben existir
df = df.dropna(subset=["Pond T-P", "Pond T-S"], how="any").copy()

# 3) Mantener solo estaciones objetivo
df = df[df["Estacion"].isin(ESTACIONES_OBJ)].copy()

# 4) Bins temporales
df["time_bin"] = df["T-ini"].dt.to_period(FREQ).dt.start_time

# 5) Separar por estacion
by_est = {est: df[df["Estacion"] == est].copy() for est in ESTACIONES_OBJ}

# 6) Seleccion para estaciones pequenas: sin criterio temporal, tomar todo hasta TARGET
selecciones = {}
for est in ESTACIONES_OBJ:
  dfe = by_est[est]
  if est in SMALL_STATIONS:
    if len(dfe) <= TARGET_POR_EST:
      selecciones[est] = dfe.copy()
    else:
      selecciones[est] = dfe.sample(n=TARGET_POR_EST, random_state=rng)  # por si hay mas de 200
  else:
    selecciones[est] = dfe.iloc[0:0].copy()  # placeholder para grandes

# 7) Construir el conjunto de bins a priorizar según PORT y UVER (fechas similares)
bins_small = set(pd.concat([selecciones[s] for s in SMALL_STATIONS if s in selecciones and len(selecciones[s]) > 0])["time_bin"].unique())
bins_small = sorted([b for b in bins_small if pd.notna(b)])

# Si no hay bins en las pequenas (muy extremo), usar los bins mas frecuentes del conjunto total
if len(bins_small) == 0:
  bins_small = list(df["time_bin"].value_counts().index[:10])  # top 10 como fallback

# 8) Asignar cupos a estaciones grandes (LABR, CUSH, CHAI)
def cuotas_por_bin(total_objetivo, num_bins):
  base = total_objetivo // num_bins if num_bins > 0 else 0
  resto = total_objetivo % num_bins if num_bins > 0 else 0
  cuotas = np.full(max(num_bins,1), base, dtype=int)
  cuotas[:resto] += 1
  return cuotas

for est in ESTACIONES_OBJ:
  if est in SMALL_STATIONS:
    continue
  dfe = by_est[est].copy()
  # objetivo por estacion
  objetivo = TARGET_POR_EST

  # ya hay seleccion para est? (deberia estar vacia para grandes)
  sel_existente = selecciones.get(est, dfe.iloc[0:0].copy())

  faltan = max(0, objetivo - len(sel_existente))
  if faltan == 0:
    continue

  # 8a) Intentar cubrir faltantes priorizando bins_small
  dfe_restante = dfe.drop(sel_existente.index)
  dfe_prior = dfe_restante[dfe_restante["time_bin"].isin(bins_small)].copy()

  if len(dfe_prior) >= faltan:
    # muestrear directamente dentro de bins_small
    # para repartir mejor, distribuimos cuotas equitativas por bin
    bins_ord = sorted(dfe_prior["time_bin"].unique())
    cuotas = cuotas_por_bin(faltan, len(bins_ord))
    trozos = []
    for bin_val, cuota in zip(bins_ord, cuotas):
      cand = dfe_prior[dfe_prior["time_bin"] == bin_val]
      if cuota <= 0 or len(cand) == 0:
        continue
      if len(cand) <= cuota:
        trozos.append(cand)
      else:
        trozos.append(cand.sample(n=cuota, random_state=rng))
    sel_prior = pd.concat(trozos) if len(trozos) else dfe_prior.iloc[0:0]
    # si por redondeos quedo corto, completar dentro de dfe_prior
    if len(sel_prior) < faltan:
      faltan2 = faltan - len(sel_prior)
      cand_extra = dfe_prior.drop(sel_prior.index)
      if len(cand_extra) > 0:
        extra = cand_extra.sample(n=min(faltan2, len(cand_extra)), random_state=rng)
        sel_prior = pd.concat([sel_prior, extra])
    sel_prior = sel_prior.head(faltan)
    selecciones[est] = pd.concat([sel_existente, sel_prior])
  else:
    # tomar todo lo que haya en bins_small y luego completar del resto
    sel_prior = dfe_prior
    faltan2 = faltan - len(sel_prior)
    dfe_rest_extra = dfe_restante.drop(sel_prior.index)
    if len(dfe_rest_extra) > 0:
      if len(dfe_rest_extra) <= faltan2:
        sel_extra = dfe_rest_extra
      else:
        sel_extra = dfe_rest_extra.sample(n=faltan2, random_state=rng)
    else:
      sel_extra = dfe_rest_extra.iloc[0:0]
    selecciones[est] = pd.concat([sel_existente, sel_prior, sel_extra]).head(objetivo)

# 9) Si PORT/UVER no alcanzan 200, completar sus faltantes repartiendo entre grandes
grandes = [e for e in ESTACIONES_OBJ if e not in SMALL_STATIONS]
pool_grandes = pd.concat([by_est[g].drop(selecciones[g].index) for g in grandes]).copy()

for est in SMALL_STATIONS:
  objetivo = TARGET_POR_EST
  sel_est = selecciones[est]
  faltan = max(0, objetivo - len(sel_est))
  if faltan == 0:
    continue

  # priorizar completar desde bins_small (ya definido por pequenas)
  pool_prior = pool_grandes[pool_grandes["time_bin"].isin(bins_small)]
  tomar = min(faltan, len(pool_prior))
  if tomar > 0:
    add = pool_prior.sample(n=tomar, random_state=rng)
    selecciones[est] = pd.concat([sel_est, add])
    pool_grandes = pool_grandes.drop(add.index)
    faltan -= tomar

  # si aun faltan, completar del pool restante
  if faltan > 0 and len(pool_grandes) > 0:
    add2 = pool_grandes.sample(n=min(faltan, len(pool_grandes)), random_state=rng)
    selecciones[est] = pd.concat([selecciones[est], add2])
    pool_grandes = pool_grandes.drop(add2.index)

# 10) Ensamblar dataset final
df_sampled = pd.concat([selecciones[e] for e in ESTACIONES_OBJ], ignore_index=True)

# 11) Validaciones basicas
print("Conteo por estacion (esperado hasta 200 c/u, PORT/UVER pueden incluir completados de otras):")
print(df_sampled["Estacion"].value_counts())

print("\nMuestreo unico esperado 64.0 ->", df_sampled["Muestreo"].unique())
na_tp = df_sampled["Pond T-P"].isna().sum()
na_ts = df_sampled["Pond T-S"].isna().sum()
print(f"NA en Pond T-P: {na_tp} | NA en Pond T-S: {na_ts}")

# 12) Guardar
df_sampled.to_csv(OUT_CSV, index=False)
print("\nGuardado en:", OUT_CSV)


**Nueva version que controla que T-P, T-S > T_ini**

In [ ]:
import pandas as pd
import numpy as np

# ========= Rutas en Colab =========
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
IN_CSV  = '/content/drive/MyDrive/Colab Notebooks/TFM/test.csv'
OUT_CSV = '/content/drive/MyDrive/Colab Notebooks/TFM/dataset_estratificado_1000.csv'
# ==================================

# ========= Parametros =========
FREQ = "W"                 # "D" dia, "W" semana, "M" mes
SEED = 42
TARGET_POR_EST = 200
ESTACIONES_OBJ = ["LABR", "CUSH", "CHAI", "PORT", "UVER"]
SMALL_STATIONS = {"PORT", "UVER"}
# ==============================

rng = np.random.RandomState(SEED)

# 1) Cargar y preparar
df = pd.read_csv(IN_CSV, sep=";")

# Estacion
df.insert(0, "Estacion", df["mseed"].str.split("_").str[0])

# Tiempos - convertir todas las columnas temporales
df["T-ini"] = pd.to_datetime(df["T-ini"], errors="coerce", utc=True)
df["T-fin"] = pd.to_datetime(df["T-fin"], errors="coerce", utc=True)
df["T-P"] = pd.to_datetime(df["T-P"], errors="coerce", utc=True)
df["T-S"] = pd.to_datetime(df["T-S"], errors="coerce", utc=True)

# 2) Filtros estrictos requeridos
df = df[df["Muestreo"] == 64.0].copy()

# Sin NA en ponderaciones: ambas deben existir
df = df.dropna(subset=["Pond T-P", "Pond T-S"], how="any").copy()

# NUEVO FILTRO: T-P > T-ini y T-S > T-ini
print(f"Registros antes del filtro temporal: {len(df)}")
df = df[(df["T-P"] > df["T-ini"]) & (df["T-S"] > df["T-ini"])].copy()
print(f"Registros después del filtro temporal (T-P>T-ini y T-S>T-ini): {len(df)}")

# 3) Mantener solo estaciones objetivo
df = df[df["Estacion"].isin(ESTACIONES_OBJ)].copy()
print(f"Registros después de filtro por estaciones objetivo: {len(df)}")

# 4) Bins temporales
df["time_bin"] = df["T-ini"].dt.to_period(FREQ).dt.start_time

# 5) Separar por estacion
by_est = {est: df[df["Estacion"] == est].copy() for est in ESTACIONES_OBJ}

# Mostrar disponibilidad por estación después de todos los filtros
print("\nRegistros disponibles por estación después de filtros:")
for est in ESTACIONES_OBJ:
    print(f"{est}: {len(by_est[est])} registros")

# 6) Seleccion para estaciones pequenas: sin criterio temporal, tomar todo hasta TARGET
selecciones = {}
for est in ESTACIONES_OBJ:
  dfe = by_est[est]
  if est in SMALL_STATIONS:
    if len(dfe) <= TARGET_POR_EST:
      selecciones[est] = dfe.copy()
    else:
      selecciones[est] = dfe.sample(n=TARGET_POR_EST, random_state=rng)  # por si hay mas de 200
  else:
    selecciones[est] = dfe.iloc[0:0].copy()  # placeholder para grandes

# 7) Construir el conjunto de bins a priorizar según PORT y UVER (fechas similares)
bins_small = set(pd.concat([selecciones[s] for s in SMALL_STATIONS if s in selecciones and len(selecciones[s]) > 0])["time_bin"].unique())
bins_small = sorted([b for b in bins_small if pd.notna(b)])

# Si no hay bins en las pequenas (muy extremo), usar los bins mas frecuentes del conjunto total
if len(bins_small) == 0:
  bins_small = list(df["time_bin"].value_counts().index[:10])  # top 10 como fallback

# 8) Asignar cupos a estaciones grandes (LABR, CUSH, CHAI)
def cuotas_por_bin(total_objetivo, num_bins):
  base = total_objetivo // num_bins if num_bins > 0 else 0
  resto = total_objetivo % num_bins if num_bins > 0 else 0
  cuotas = np.full(max(num_bins,1), base, dtype=int)
  cuotas[:resto] += 1
  return cuotas

for est in ESTACIONES_OBJ:
  if est in SMALL_STATIONS:
    continue
  dfe = by_est[est].copy()
  # objetivo por estacion
  objetivo = TARGET_POR_EST

  # ya hay seleccion para est? (deberia estar vacia para grandes)
  sel_existente = selecciones.get(est, dfe.iloc[0:0].copy())

  faltan = max(0, objetivo - len(sel_existente))
  if faltan == 0:
    continue

  # 8a) Intentar cubrir faltantes priorizando bins_small
  dfe_restante = dfe.drop(sel_existente.index)
  dfe_prior = dfe_restante[dfe_restante["time_bin"].isin(bins_small)].copy()

  if len(dfe_prior) >= faltan:
    # muestrear directamente dentro de bins_small
    # para repartir mejor, distribuimos cuotas equitativas por bin
    bins_ord = sorted(dfe_prior["time_bin"].unique())
    cuotas = cuotas_por_bin(faltan, len(bins_ord))
    trozos = []
    for bin_val, cuota in zip(bins_ord, cuotas):
      cand = dfe_prior[dfe_prior["time_bin"] == bin_val]
      if cuota <= 0 or len(cand) == 0:
        continue
      if len(cand) <= cuota:
        trozos.append(cand)
      else:
        trozos.append(cand.sample(n=cuota, random_state=rng))
    sel_prior = pd.concat(trozos) if len(trozos) else dfe_prior.iloc[0:0]
    # si por redondeos quedo corto, completar dentro de dfe_prior
    if len(sel_prior) < faltan:
      faltan2 = faltan - len(sel_prior)
      cand_extra = dfe_prior.drop(sel_prior.index)
      if len(cand_extra) > 0:
        extra = cand_extra.sample(n=min(faltan2, len(cand_extra)), random_state=rng)
        sel_prior = pd.concat([sel_prior, extra])
    sel_prior = sel_prior.head(faltan)
    selecciones[est] = pd.concat([sel_existente, sel_prior])
  else:
    # tomar todo lo que haya en bins_small y luego completar del resto
    sel_prior = dfe_prior
    faltan2 = faltan - len(sel_prior)
    dfe_rest_extra = dfe_restante.drop(sel_prior.index)
    if len(dfe_rest_extra) > 0:
      if len(dfe_rest_extra) <= faltan2:
        sel_extra = dfe_rest_extra
      else:
        sel_extra = dfe_rest_extra.sample(n=faltan2, random_state=rng)
    else:
      sel_extra = dfe_rest_extra.iloc[0:0]
    selecciones[est] = pd.concat([sel_existente, sel_prior, sel_extra]).head(objetivo)

# 9) Si PORT/UVER no alcanzan 200, completar sus faltantes repartiendo entre grandes
grandes = [e for e in ESTACIONES_OBJ if e not in SMALL_STATIONS]
pool_grandes = pd.concat([by_est[g].drop(selecciones[g].index) for g in grandes]).copy()

for est in SMALL_STATIONS:
  objetivo = TARGET_POR_EST
  sel_est = selecciones[est]
  faltan = max(0, objetivo - len(sel_est))
  if faltan == 0:
    continue

  # priorizar completar desde bins_small (ya definido por pequenas)
  pool_prior = pool_grandes[pool_grandes["time_bin"].isin(bins_small)]
  tomar = min(faltan, len(pool_prior))
  if tomar > 0:
    add = pool_prior.sample(n=tomar, random_state=rng)
    selecciones[est] = pd.concat([sel_est, add])
    pool_grandes = pool_grandes.drop(add.index)
    faltan -= tomar

  # si aun faltan, completar del pool restante
  if faltan > 0 and len(pool_grandes) > 0:
    add2 = pool_grandes.sample(n=min(faltan, len(pool_grandes)), random_state=rng)
    selecciones[est] = pd.concat([selecciones[est], add2])
    pool_grandes = pool_grandes.drop(add2.index)

# 10) Ensamblar dataset final
df_sampled = pd.concat([selecciones[e] for e in ESTACIONES_OBJ], ignore_index=True)

# 11) Validaciones basicas
print("\n" + "="*50)
print("RESULTADOS FINALES")
print("="*50)
print("Conteo por estacion (esperado hasta 20 c/u, PORT/UVER pueden incluir completados de otras):")
print(df_sampled["Estacion"].value_counts())

print(f"\nMuestreo unico esperado 64.0 -> {df_sampled['Muestreo'].unique()}")
na_tp = df_sampled["Pond T-P"].isna().sum()
na_ts = df_sampled["Pond T-S"].isna().sum()
print(f"NA en Pond T-P: {na_tp} | NA en Pond T-S: {na_ts}")

# Validación adicional del filtro temporal aplicado
violaciones_tp = (df_sampled["T-P"] <= df_sampled["T-ini"]).sum()
violaciones_ts = (df_sampled["T-S"] <= df_sampled["T-ini"]).sum()
print(f"\nValidación filtro temporal:")
print(f"Violaciones T-P <= T-ini: {violaciones_tp} (debe ser 0)")
print(f"Violaciones T-S <= T-ini: {violaciones_ts} (debe ser 0)")

# 12) Formatear tiempos antes de guardar
def format_datetime_iso(dt_series):
    """Convierte datetime a formato ISO 8601 con milisegundos y zona UTC"""
    return dt_series.dt.strftime('%Y-%m-%dT%H:%M:%S.%f').str[:-3] + 'Z'

# Crear copia para formateo sin afectar los datos originales
df_output = df_sampled.copy()

# Formatear todas las columnas de tiempo
time_columns = ['T-ini', 'T-fin', 'T-P', 'T-S']
for col in time_columns:
    if col in df_output.columns:
        df_output[col] = format_datetime_iso(df_output[col])

print(f"\nEjemplo de formato de tiempos en salida:")
print(f"T-ini: {df_output['T-ini'].iloc[0]}")
print(f"T-P: {df_output['T-P'].iloc[0]}")
print(f"T-S: {df_output['T-S'].iloc[0]}")

# Guardar con formato preservado
df_output.to_csv(OUT_CSV, index=False)
print(f"\nGuardado en: {OUT_CSV}")
print(f"Total de registros en dataset final: {len(df_sampled)}")